In [ ]:
from datasets import Dataset, Image, load_dataset
import os
from os import listdir
from os.path import isfile, join
import json
from datetime import date
# define folder name
folder_name = "all_annotations_adjusted_2"
# define load_dataset paths
img_path = "/home/mcontini/Desktop/PhD/panoptic_51_leila_kaneda/" + folder_name + "/images/data/"
label_path = "/home/mcontini/Desktop/PhD/panoptic_51_leila_kaneda/" + folder_name + "/panoptic_segs/"
panoptic_segmentation_json_path = "/home/mcontini/Desktop/PhD/panoptic_51_leila_kaneda/" + folder_name + "/" + "panoptic_adjusted.json"
# define "today" date
today = date.today()
today = today.strftime("%Y_%m_%d")

In [ ]:
# Opening JSON file
f = open(panoptic_segmentation_json_path)
# returns JSON object as a dictionary
data = json.load(f)
# define list of polygons
list_of_segments = []
# define list of images names
list_of_img_names = []
for i in data['annotations']:
    # get image name without the extension (jpg / png etc)
    list_of_img_names.append(os.path.splitext(i["file_name"])[0])
    # get corresponding segment_info
    list_of_segments.append(i["segments_info"])

# add home_path png extension to all the image labels
list_of_labels = [join(label_path, f) for f in list_of_img_names]
list_of_labels = [f + ".png" for f in list_of_labels]

list_of_images1 = [join(img_path, f) for f in list_of_img_names]
list_of_images = []
for f in list_of_images1 :
    if isfile(f + ".jpg") :
        list_of_images.append(f + ".jpg")
    elif isfile(f + ".JPG") :
        list_of_images.append(f + ".JPG")
    elif isfile(f + ".jpeg") :
        list_of_images.append(f + ".jpeg")

In [ ]:
# create the dict of dataset classes
data = json.load(f)
id2label = {}
for i in data["categories"] : 
    cat_nb = i["id"]
    cat_name = i["name"]
    id2label[cat_nb] = cat_name

In [ ]:
# push dataset to the hub
ds = Dataset.from_dict({"image": list_of_images, "label": list_of_labels, "segments_info": list_of_segments, 'image_name': list_of_img_names})
ds = ds.cast_column("image", Image())
ds = ds.cast_column("label", Image())
ds_name = "lombardata/panoptic_" + today
ds_name
ds.push_to_hub(ds_name)